In [1]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz -P /content/drive/MyDrive # link wrong in blog
!tar xf /content/drive/Shareddrives/DA231-2021-Aug-Public/spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

Mounted at /content/drive


In [5]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
         .master("local")\
         .appName("Colab")\
         .config('spark.ui.port', '4050')\
         .getOrCreate()
spark

from pyspark import SparkContext, SparkConf
import sys
import json
#from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.types import *
#from pyspark.sql.functions import *
import pyspark.sql.functions as F
sc = SparkContext.getOrCreate()

In [6]:
# list of 182 users
num_enabled_users = 10
user_list = list(range(num_enabled_users))
if (128 in user_list):
  user_list.remove(128)
# define schema
schema = StructType([
    StructField("Lat", StringType(), True), StructField("Long", StringType(), True), StructField("Zero", StringType(), True), StructField("Alt", StringType(), True), StructField("DayCount", StringType(), True), StructField("Date", StringType(), True), StructField("Time", StringType(), True), StructField("UserId", StringType(), True)
])

# empty combined df
df_combine = spark.createDataFrame([], schema)


for ii in range(len(user_list)):
  folder_name = "{0:0=3d}".format(user_list[ii])
  data_user = "/content/drive/MyDrive/DataEngAtScaleProject/GeolifeTrajectories/Data/"+folder_name+"/Trajectory/*.plt"
  #data_user = "/content/drive/MyDrive/DataEngAtScaleProject/GeolifeTrajectories/Data/000/Trajectory/20081023025304.plt"
  #data000 = "/content/drive/MyDrive/DataEngAtScaleProject/GeolifeTrajectories/Data/000/Trajectory/20081023025304.plt"

  # read as rdd
  read_user = sc.textFile(data_user)

  # remove header lines and split the values
  rdd_user = read_user.filter(lambda x: len(x)>40).map(lambda l :  l.split(","))

  # convert rdd to dataframe
  df_user = spark.createDataFrame(rdd_user).toDF("Lat","Long","Zero","Alt","DayCount","Date","Time").withColumn("UserId",F.lit(folder_name))
  df_combine = df_combine.union(df_user)

df_combine = df_combine.select(F.col("Lat").cast(DoubleType()).alias("Lat"), F.col("Long").cast(DoubleType()).alias("Long"), F.col("Alt").cast(IntegerType()).alias("Alt"), F.col("DayCount").cast(DoubleType()).alias("DayCount"), F.col("Date").cast(DateType()).alias("Date"), "Time", "UserId").cache()

#print(df_combine.take(5))
#print(df_combine.count())

In [7]:
# UDF
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

def GetCityName(Coord):
  location = geolocator.reverse(Coord, language='en')
  address = location.raw['address']
  city = address.get('city', '')
  return city

GetCityName_UDF = F.udf(GetCityName, StringType())

In [ ]:
AggregateByDay = df_combine.withColumn("Coord",F.concat_ws(",",("Lat"),("Long"))).select("Coord","Date","UserId").groupBy("Date","UserId").agg(F.first("Coord").alias("Coord")).cache()
CityUserDate = AggregateByDay.withColumn("CityName",GetCityName_UDF(F.col("Coord"))).select("CityName","UserId","Date")
CityList = CityUserDate.select("CityName").distinct().collect()
CityListFile = open("/content/drive/MyDrive/DataEngAtScaleProject/CityListOutputLog.txt", "w")
for row in CityList:
  CityListFile.writelines(row["CityName"] + ",")

CityListFile.writelines('\n')
CityUser = CityUserDate.groupBy("UserId","CityName").count().groupBy("UserId").agg(F.max(F.struct(F.col("count"),F.col("CityName"))).alias("max")).select(F.col("UserId"), F.col("max.CityName")).cache()
CityUserSorted = CityUser.withColumn("UserNo",CityUser["UserId"].cast(IntegerType())).drop("UserId").sort("UserNo").collect()
import csv
with open('/content/drive/MyDrive/DataEngAtScaleProject/CityAllUserSorted.csv', 'w') as f:
    for row in CityUserSorted:
        f.write("%d,%s,%s\n"%(row["UserNo"],",",row["CityName"]))



In [ ]:
Most_Recorded_common_days = AggregateByDay.select("Date").groupBy("Date").count()
Most_Recorded_common_days.toPandas().to_csv("/content/drive/MyDrive/DataEngAtScaleProject/CommonRecordedDays.txt")
CommonRecordedMonths = Most_Recorded_common_days.withColumn("YearMonth",F.concat_ws("-",F.split("Date","-").getItem(0),F.split("Date","-").getItem(1))).select("YearMonth","count").groupBy("YearMonth").agg(F.sum("count").alias("NumUserDays")).sort("NumUserDays",ascending = False)
#CommonRecordedMonths.toPandas().to_csv("/content/drive/MyDrive/DataEngAtScaleProject/CommonRecordedMonths.txt")

In [ ]:
AggregateByMinutes = df_combine.where(F.col("Date")>="2008-05-01").where(F.col("Date")<="2009-07-31").select("Lat","Long","Date","UserId",F.concat_ws(":",F.split("Time",":").getItem(0),F.split(F.split("Time",":").getItem(1),'').getItem(0)).alias("Minutes")).groupBy("Date","Minutes","UserId").agg(F.first("Lat").alias("Lat"),F.first("Long").alias("Long")).cache()
LessPreciseCoord10MinInterval = AggregateByMinutes.withColumn("ShortLat",AggregateByMinutes.Lat.cast(DecimalType(scale=5))).withColumn("ShortLong",AggregateByMinutes.Long.cast(DecimalType(scale=5))).drop("Lat").drop("Long").cache()
#LessPreciseCoord10MinInterval.toPandas().to_csv("/content/drive/MyDrive/DataEngAtScaleProject/LessPreciseCoord10MinInterval.txt")

In [ ]:
LessPreciseCoord10MinInterval = spark.read.option("header",True).csv("/content/drive/MyDrive/DataEngAtScaleProject/LessPreciseCoord10MinInterval.txt")
LessPreciseCoord10MinInterval = LessPreciseCoord10MinInterval.select("UserId",F.col("ShortLat").cast(DoubleType()).alias("ShortLat"), F.col("ShortLong").cast(DoubleType()).alias("ShortLong"),F.col("Date").cast(DateType()).alias("Date"),"Minutes").cache()
LessPreciseCoord10MinInterval.printSchema()

In [ ]:

UserListForSocRel = LessPreciseCoord10MinInterval.select("UserId").distinct().sort("UserId").collect()
Per10MinData_user = {}
for ii in range(len(UserListForSocRel)):
  user_name = UserListForSocRel[ii][0]
  Per10MinData_user[user_name] = LessPreciseCoord10MinInterval.where(F.col("UserId")==user_name).cache()

separationPerIter = open("/content/drive/MyDrive/DataEngAtScaleProject/perIterSeparationLog.txt", "a")
separationPerIter.writelines("New Run Starts Here")
separationPerIter.writelines('\n')

for ii in range(len(UserListForSocRel)):
  SeparationTable = {}
  CountTable = {}
  for jj in range(ii+1,len(UserListForSocRel)):
    user_name1 = UserListForSocRel[ii][0]
    user_name2 = UserListForSocRel[jj][0]
    #if CityUserSorted[ii][0] == CityUserSorted[jj][0]:
    df1 = Per10MinData_user[user_name1].withColumnRenamed("ShortLat","Lat1").withColumnRenamed("ShortLong","Long1")
    df2 = Per10MinData_user[user_name2].withColumnRenamed("ShortLat","Lat2").withColumnRenamed("ShortLong","Long2")
    joinDF = df1.join(df2,["Date","Minutes"]).cache()
    commonTimeCount = joinDF.count()
    if commonTimeCount>50:   #atleast consider 50 common timestamps
      tempVar = joinDF.withColumn("AbsDiff",F.abs(F.col("Lat1")-F.col("Lat2"))+F.abs(F.col("Long1")-F.col("Long2"))).select("AbsDiff").agg(F.avg("AbsDiff").alias("MeanSeparation")).collect()[0][0]
      SeparationTable[(user_name1,user_name2)] = float(tempVar)
      CountTable[(user_name1,user_name2)] = commonTimeCount
  if bool(SeparationTable):
    ClosestUsers = min(SeparationTable, key=SeparationTable.get)
    separationPerIter.writelines(str(ClosestUsers))
    separationPerIter.writelines(":")
    separationPerIter.writelines(str(min(SeparationTable.values())))
    separationPerIter.writelines(":")
    separationPerIter.writelines(str(CountTable[ClosestUsers]))    
    separationPerIter.writelines("\n")

separationPerIter.close()
